## NetCDF data

In [1]:
# %matplotlib inline
# Import the tools we are going to need today:
import os
import datetime

import numpy as np  # numerical library
import pandas as pd
import xarray as xr  # netCDF library
from scipy import signal
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import netCDF4
import matplotlib.pyplot as plt  # plotting library
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from matplotlib.patches import Rectangle
from matplotlib.colors import BoundaryNorm, LogNorm, LinearSegmentedColormap

import warnings

import xrft
# Some defaults:
# np.set_printoptions(threshold=50)  # avoid to print very large arrays on screen

# change to eratools and move it
from eratools.eratools import era_ds_processor, era_filter, era_cmaps

import LidarT_visualization

plt.style.use('latex_default.mplstyle')

In [2]:
# --- Download of ERA5 data --- #
ml_coeff = pd.read_csv('data/era5-ml-coeff.csv')

#era_case = '2014-07-16'
#era_case = '2020-08-07'
era_case = '2018-06-22' # done

#ds      = xr.open_dataset('data/ERA5-' + era_case + '-ml.nc')
#ds_T21  = xr.open_dataset('data/ERA5-' + era_case + '-ml-T21.nc')
ds_pv   = xr.open_dataset('data/ERA5-' + era_case + '-pl.nc')
ds_2pvu = xr.open_dataset('data/ERA5-' + era_case + '-pvu.nc')

#ds_interp_path = 'data/ERA5-2014-07-16-ml-interp-T42.nc'
ds_interp_path = 'data/ERA5-' + era_case + '-ml-interp.nc'
use_interpolated_ds = True

output_folder = 'output/' + era_case + '/'

if era_case == '2014-07-16':
    lat = -53.75
    lon = 140
    lat_avg = [-52.5,-57.5] # -53.7
    lon_avg = [147.5,152.5] # [290,295] # -67.8 # !!!use eastern coordinates!!!
    western_coordinates    = False
    plot_lidar_measurement = False
    lon_range = [90,180]
    lon_eastern = lon
    lidar_label = 'lidar'
else:
    lat = -53.75
    lon = -67.75
    lat_avg = [-52.5,-57.5] # -53.7
    lon_avg = [290,295] # -67.8 # !!!use eastern coordinates!!!
    western_coordinates    = True
    plot_lidar_measurement = True
    lon_range = [-120,-30]
    lon_eastern = 360+lon
    lidar_label = 'CORAL'

lat_range = [-77.5,-32.5]

# lat_range = [-52.5,-57.5] # -53.7
# lon_range_pv = [147.5,152.5]  # [290,295] # -67.8
# lon_range = lon_range_pv
# lon_range = np.array(lon_range_pv) + 360 # -67.8

#### - Preprocessing of ERA5 data - ####
if use_interpolated_ds==False:
    ds = era_ds_processor.prepare_interpolated_ml_ds(ds,ds_T21,ml_coeff,ds_interp_path)
else:
    ds = xr.open_dataset(ds_interp_path)
ds

<xarray.Dataset>
Dimensions:    (time: 72, level: 176, latitude: 241, longitude: 361)
Coordinates:
  * time       (time) datetime64[ns] 2018-06-22 ... 2018-06-24T23:00:00
  * level      (level) float64 0.0 400.0 800.0 ... 6.92e+04 6.96e+04 7e+04
  * latitude   (latitude) float32 -25.0 -25.25 -25.5 ... -84.5 -84.75 -85.0
  * longitude  (longitude) float32 240.0 240.2 240.5 240.8 ... 329.5 329.8 330.0
Data variables:
    t          (time, level, latitude, longitude) float32 ...
    p          (time, level, latitude, longitude) float32 ...
    u          (time, level, latitude, longitude) float32 ...
    v          (time, level, latitude, longitude) float32 ...
    tprime     (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-10 14:56:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [3]:
#### - Further processing of ERA5 data - ####
ds,ds_pv,ds_2pvu = era_ds_processor.processing_data_for_jetexit_comp(ds,ds_pv,ds_2pvu,western_coordinates)

In [4]:
"""Averaging lat lon bands"""
#th_lon_z = ds['th'].sel(latitude =slice(lat_range[0],lat_range[1])).mean(axis=2).copy()
#th_lat_z = ds['th'].sel(longitude=slice(lon_range[0],lon_range[1])).mean(axis=3).copy()
# - Or select single lat/lon band - #
th_lon_z = ds['th'].sel(latitude=lat)
th_lat_z = ds['th'].sel(longitude=lon_eastern)

n2_lon_z = ds['N2'].sel(latitude =slice(lat_avg[0],lat_avg[1])).mean(axis=2)
n2_lat_z = ds['N2'].sel(longitude=slice(lon_avg[0],lon_avg[1])).mean(axis=3)

#v_lon_z = ds['v'].sel(latitude =slice(lat_range[0],lat_range[1])).mean(axis=2).copy()
#u_lat_z = ds['u'].sel(longitude=slice(lon_range[0],lon_range[1])).mean(axis=3).copy()
v_lon_z  = ds['v'].sel(latitude=lat)
u_lat_z  = ds['u'].sel(longitude=lon_eastern)
UV_lon_z = (ds['u'].sel(latitude=lat)**2 + ds['v'].sel(latitude=lat)**2)**(1/2)
UV_lat_z = (ds['u'].sel(longitude=lon_eastern)**2 + ds['v'].sel(longitude=lon_eastern)**2)**(1/2)

# - Rolling mean - #
# tmp_mean = ds['t'].rolling(longitude=90, center = True).mean(dim='longitude')
# ds['tprime_runningM'] = ds['t'].copy()-tmp_mean

# mean_temp = ds['t'].mean(axis=3).copy()
# ds['tprime'] = ds['t'].copy() - mean_temp

### - PV dataset (on pressure levels) - ###
pv_lon_z = ds_pv['pv'].sel(latitude =slice(lat_avg[0],lat_avg[1])).mean(axis=2) * 10**(6)
pv_lat_z = ds_pv['pv'].sel(longitude=slice(lon_avg[0],lon_avg[1])).mean(axis=3) * 10**(6)

zpv_lon_z = ds_pv['z'].sel(latitude =slice(lat_avg[0],lat_avg[1])).mean(axis=2)
zpv_lat_z = ds_pv['z'].sel(longitude=slice(lon_avg[0],lon_avg[1])).mean(axis=3)

In [5]:
###### CORAL DATA #######
if plot_lidar_measurement:
    fileLocation = 'data-coral'
    if era_case == '2020-08-07':
        fileNames = ["20200807-2156_T15Z900.nc", "20200808-2156_T15Z900.nc"]
        timeframe = [21,10]
    else:
        fileNames = ["20180622-2309_T20Z900.nc"] # 1
        timeframe = [23,12] # 1

    SETTINGS_FILE = 'coral-visualization/settings_coral.txt'
    SETTINGS = {}
    with open(SETTINGS_FILE, 'r') as file:
        for line in file:
            try:
                line = line.strip()
                (key, val) = line.split(": ")
                SETTINGS[key] = val
            except:
                print('The following line could not be executed: ' + line)
                print('Variable might be missing!')

    # fig = LidarT_visualization.plot_LidarT(file_location=fileLocation, file_name=fileName, SETTINGS=SETTINGS, plot_content='butterworthF', fixed_timeframe=True, stats_content='nightly_mean', stats_path='', save_fig=True)
    coral_measurements = []
    for fileName in fileNames:
        path = os.path.join(fileLocation, fileName)
        DS = xr.open_dataset(path, decode_times=False)

        DS.coords['time'] = DS.time.values / 1000
        DS.integration_start_time.values = DS.integration_start_time.values / 1000
        DS.integration_end_time.values = DS.integration_end_time.values / 1000

        # Decode times with time offset
        unit_str = DS.time_offset.attrs['units']
        DS.attrs['reference_date'] = unit_str[14:-6]
        # Reference date is first reference
        # 'Time offset' is 'seconds' after reference date
        # Time is 'seconds' after time offset

        time_reference = datetime.datetime.strptime(DS.reference_date, '%Y-%m-%d %H:%M:%S.%f')
        time_offset = datetime.timedelta(seconds=float(DS.time_offset.values[0]))
        new_time_reference = time_reference + time_offset
        time_reference_str = datetime.datetime.strftime(new_time_reference, '%Y-%m-%d %H:%M:%S')

        DS.time.attrs['units'] = 'seconds since ' + time_reference_str
        DS.integration_start_time.attrs['units'] = 'seconds since ' + time_reference_str
        DS.integration_end_time.attrs['units'] = 'seconds since ' + time_reference_str

        DS = xr.decode_cf(DS, decode_coords = True, decode_times = True) 

        fixed_start = timeframe[0]
        fixed_end = timeframe[1]
        if fixed_end < fixed_start:
            fixed_intervall = fixed_end + 24 - fixed_start
        else: 
            fixed_intervall = fixed_end - fixed_start
            
        start_date = datetime.datetime.utcfromtimestamp(DS.time.values[0].astype('O')/1e9)
        fixed_start_date = datetime.datetime(start_date.year, start_date.month, start_date.day, fixed_start, 0,0)
        duration = datetime.datetime.utcfromtimestamp(DS.integration_end_time.values[-1].astype('O')/1e9) -  datetime.datetime.utcfromtimestamp(DS.integration_start_time.values[0].astype('O')/1e9)# for calendar

        reference_hour = 15
        if (start_date.hour > reference_hour) and (fixed_start_date.hour > reference_hour):
            DS['date_startp'] = fixed_start_date
            DS['date_endp'] = fixed_start_date + datetime.timedelta(hours=fixed_intervall)
        elif (start_date.hour > reference_hour) and (fixed_start_date.hour < reference_hour): # prob in range of 0 to 10
            DS['date_startp'] = fixed_start_date + datetime.timedelta(hours=24)
            DS['date_endp'] = fixed_start_date + datetime.timedelta(hours=24+fixed_intervall)
        elif (start_date.hour < reference_hour) and (fixed_start_date.hour > reference_hour):
            DS['date_startp'] = fixed_start_date - datetime.timedelta(hours=24)
            DS['date_endp'] = fixed_start_date - datetime.timedelta(hours=24-fixed_intervall)
        else: # (start_date.hour < 18) and (fixed_start_date.hour < 18):
            DS['date_startp'] = fixed_start_date
            DS['date_endp'] = fixed_start_date - datetime.timedelta(hours=fixed_intervall)
            
        DS['fixed_timeframe'] = 1

        # Temperature (Change 0 to NaN)
        DS.temperature.values = np.where(DS.temperature == 0, np.nan, DS.temperature)
        DS.temperature_err.values = np.where(DS.temperature_err == 0, np.nan, DS.temperature_err)

        DS['alt_plot'] = (DS.altitude + DS.altitude_offset + DS.station_height) / 1000 #km

        # DS = LidarT_visualization.butterworthf(DS)
        DS = LidarT_visualization.butterworthf(DS, highcut=1/15, fs=1/0.1, order=5, single_column_filter=True)

        tmp_mean = DS.temperature.rolling(time=30, center = True).mean()
        DS['tmp_runningM'] = DS.temperature-tmp_mean

        tmp_mean = DS.temperature.mean(dim='time')
        DS['tmp_tfilter'] = DS.temperature-tmp_mean

        coral_measurements.append(DS)
###### CORAL DATA #######

In [6]:
%%capture
warnings.simplefilter(action='ignore', category=FutureWarning)
g = 9.80665
plot_rectangle = False
animation = True

def major_formatter_lon(x, pos):
    if western_coordinates:
        return "%.f°W" % abs(x)
    else:
        return "%.f°E" % abs(x)

def major_formatter_lat(x, pos):
    return "%.f°S" % abs(x)

###### - Lat-z cross section of tropo and stratosphere - ######
def era5_cross_section_plot(ds,t,folder,lat,lon):
    # - FILTER stratosphere T - #
    vert_res = ds['level'].values[1]-ds['level'].values[0]

    # - LIDAR plot - #
    data_temp = ds['t'].sel(latitude=lat,longitude=lon_eastern,method='nearest').copy()
    #tprime_time = data_temp - data_temp.mean(axis=0)
    tprime_T21     = ds['tprime'].sel(latitude=lat,longitude=lon,method='nearest').values.T.copy()
    tprime_BW, tbg_BW = era_filter.butterworth_filter(data_temp.values, highcut=1/15, fs=1/(vert_res/1000), order=5)
    tprime_12hmean = data_temp - data_temp.rolling(time=12,center=True).mean()
    #tprime_12hmean = (data_temp-tprime_BW) - (data_temp-tprime_BW).rolling(time=12,center=True).mean() ## Subtract BW filter first, then running mean
    #tprime_lid = tprime_BW.T
    #tprime_lid = tprime_T21
    tprime_lid = tprime_12hmean.T

    # - Cross sections - #
    #data_temp = ds['t'].sel(latitude=lat)[t,:,:].values.copy()# [::-1,:]
    #tprime_zonal, tbg_zonal = era_filter.butterworth_filter(data_temp.T, highcut=1/15, fs=1/(vert_res/1000), order=5)
    #data_temp = ds['t'].sel(longitude=lon)[t,:,:].values.copy()# [::-1,:]
    #tprime_meridional, tbg_meridional = era_filter.butterworth_filter(data_temp.T, highcut=1/15, fs=1/(vert_res/1000), order=5)
    
    # - Horizontal FFT filter - #
    nx_avg = 56 # 50 -> approx. lambdax=800km assuming 1°=60km, 56->900km
    tprime_lon_z, tprime_lat_z = era_filter.horizontal_temp_filter(ds,t,lat,lon_eastern,nx_avg=nx_avg)
    
    # - T21 - #
    #tprime_lon_z = ds['tprime'][t,:,:,:].sel(latitude=lat,method='nearest').values.copy()
    #tprime_lat_z = ds['tprime'][t,:,:,:].sel(longitude=lon,method='nearest').values.copy()

    # - FIGURE - #
    gskw  = {'hspace':0.06, 'wspace':0.04, 'height_ratios': [3.5,4,3,0.001,3.5], 'width_ratios': [5,5,1]} #  , 'width_ratios': [5,5]}
    #gskw2 = {'hspace':0.06, 'wspace':0.06, 'height_ratios': [3.5,4,3,0.001,3.5], 'width_ratios': [7.5,2.5,1]} #  , 'width_ratios': [5,5]}
    fig, axes = plt.subplots(5,3, figsize=(10,12), gridspec_kw=gskw)

    axes[0,2].axis('off')
    axes[1,2].axis('off')
    axes[2,2].axis('off')
    axes[3,0].axis('off')
    axes[3,1].axis('off')
    axes[3,2].axis('off')
    axes[4,2].axis('off')

    gs_top = axes[0,0].get_gridspec()
    gs = axes[3,0].get_gridspec()

    # remove the underlying axes
    for ax in axes[0,0:2]:
        ax.remove()
    # ax_lid = fig.add_subplot(gs_top[0,0:2])
    gs_top2 = fig.add_gridspec(5,3, hspace=0.06, wspace=0.04, height_ratios=[3.5,4,3,0.001,3.5], width_ratios=[8.5,1.5,1])
    ax_lid  = fig.add_subplot(gs_top2[0])
    ax_lid2 = fig.add_subplot(gs_top2[1])

    for ax in axes[-1,0:2]:
        ax.remove()

    projection = ccrs.PlateCarree()
    ax_pvu = fig.add_subplot(gs[-1,0:2], projection=projection)
    lw_cut = 2

    # - Define levels and colormaps to plot for relevant variables - #
    thlev_st = np.exp(5+0.03*np.arange(1,120,4))
    thlev_st_labels = np.exp(5+0.03*np.arange(1,120,8))
    lon_label = lon-35
    lat_label = lat-15

    thlev = np.arange(220,600,5)
    thlev_labels = np.arange(220,600,40)

    #ulev = np.arange(-180,180,20)
    ulev1 = np.arange(20,180,20)
    ulev0 = np.arange(-160,0,20)
    ulev = np.concatenate((ulev0,ulev1)) 
    lw_wind = 0.5
    lw_thin=0.1
    lw_thick=2

    pvlev = [-4,-3,-2,-1]

    CLEV_11 = [-11,-9,-7,-5,-3,-1,1,3,5,7,9,11]
    CLEV_11_LABELS = [-9,-7,-5,-3,-1,1,3,5,7,9]
    # CLEV_32 = [-32,-16,-8,-4,-2,-1,-0.5,-0.25,0.25,0.5,1,2,4,8,16,32]
    # CLEV_32_LABELS = [-16,-4,-1,-0.25,0.25,1,4,16]
    CLEV_16   = [-16,-8,-4,-2,-1,-0.5,0.5,1,2,4,8,16] # 32
    # CLEV_16   = [-16,-8,-4,-2,-1,1,2,4,8,16] # 32
    # CLEV_16_LABELS = [-16,-4,-1,1,4,16]
    CLEV_16_LABELS = [-16,-8,-4,-2,-1,1,2,4,8,16]

    clev = CLEV_16
    clev_l = CLEV_16_LABELS
    cmap_st = plt.get_cmap('RdBu_r')
    cmap_st = era_cmaps.get_wave_cmap()
    norm_st = BoundaryNorm(boundaries=clev, ncolors=cmap_st.N, clip=True)

    n2lev = np.array([-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5])
    # clev_l = [0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5]*10**-4
    cmap = plt.get_cmap('coolwarm')
    norm = BoundaryNorm(boundaries=n2lev , ncolors=cmap.N, clip=True)

    ### - LIDAR - ###
    ## pcolor_lid = ax_lid.pcolormesh(ds['time'].values, ds['level']/1000, data_temp,cmap='jet', vmin=180,vmax=300) # absolute temperature
    ## contourf_lid = ax_lid.contourf(ds['time'].values, ds['level']/1000, tprime_lid.T, levels=clev, cmap=cmap_st, norm=norm_st, extend='both')
    pcolor_lid = ax_lid.pcolormesh(ds['time'].values, ds['level']/1000, tprime_lid, cmap=cmap_st, norm=norm_st, shading='nearest') # BW

    ###### -- CORAL DATA -- ######
    if plot_lidar_measurement:
        # clev_coral = [-12,-6,6,12]
        # clev_coral = [-16,-14,-12,-10,-8,-6,6,8,10,12,14,16]
        clev_coral = [-10,-5,5,10]
        clev_coral = [-15,-10,-5,5,10,15]
        red = 'firebrick'
        blue = 'royalblue'
        coral_colors = [blue,blue,blue,red,red,red]
        # clev_coral = [-32,-16,-8,8,16,32]
        cmap_coral = plt.get_cmap('RdBu_r')
        norm_coral = BoundaryNorm(boundaries=clev_coral, ncolors=cmap_coral.N, clip=True) 
        # pcolor_1 = ax_lid.contour(DS.time.values, DS.alt_plot.altitude/1000, np.matrix.transpose(DS.tmp_pert.values),
        #                        levels=clev_coral, cmap=cmap_coral, norm=norm_coral)
        # pcolor_1 = ax_lid.contour(DS.time.values, DS.alt_plot.altitude/1000, np.matrix.transpose(DS.tmp_pert.values),
        #                         levels=clev_coral, colors='k', linewidths=0.67) # negative_linestyles='dashed'
        for DS in coral_measurements:
            pcolor_1 = ax_lid.contour(DS.time.values, DS.alt_plot.altitude/1000, np.matrix.transpose(DS.tmp_tfilter.values),
                                    levels=clev_coral, colors='k', linewidths=0.67) # negative_linestyles='dashed'
    ###### -- CORAL DATA -- #######
    
    h_fmt = mdates.DateFormatter('%b-%d %H:%M')
    ax_lid.xaxis.set_major_formatter(h_fmt)
    # ax_lid.xaxis.set_major_locator(h_interv)
    ax_lid.yaxis.set_minor_locator(AutoMinorLocator()) 
    ax_lid.xaxis.set_minor_locator(AutoMinorLocator())
    
    ax_lid.xaxis.set_label_position('top')
    ax_lid.tick_params(which='both', labelbottom=False,labeltop=True)
    ax_lid.set_ylabel('altitude / km')
    ax_lid.set_ylim(14,61)
    ax_lid.set_xlim(ds['time'].values[0],ds['time'].values[-1])
    time_ticks = ax_lid.get_xticks()
    ax_lid.set_xticks(time_ticks[1::])
    ## ax_lid.set_aspect(0.05)

    fs_clabel = 8.5
    
    if animation == True:
        ax_lid2.plot(data_temp[t,:],ds['level']/1000,lw=2*lw_thin,color='black')
    else:
        for tt in range(0,np.shape(ds['t'])[0],3):      
            ax_lid2.plot(data_temp[tt,:],ds['level']/1000,lw=lw_thin,color='black')
    
    ax_lid2.plot(np.mean(data_temp,axis=0),ds['level']/1000,lw=lw_thick,color='black')
    ax_lid2.yaxis.set_minor_locator(AutoMinorLocator()) 
    ax_lid2.xaxis.set_minor_locator(AutoMinorLocator())
    ax_lid2.xaxis.set_label_position('top')
    ax_lid2.tick_params(which='both', labelleft=False, labelbottom=False,labeltop=True)
    ax_lid2.set_ylim(14,61)
    ax_lid2.set_xlim([190,263])
    ax_lid2.xaxis.set_major_formatter("{x:.0f}K")
    ax_lid2.grid()

    timestamp_str = str(ds.time[t].values)[0:16].replace('T',' ')
    ax_lid.text(0.98, 0.955, timestamp_str, transform=ax_lid.transAxes, verticalalignment='top', horizontalalignment='right', bbox={"boxstyle" : "round", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    ax_lid.axvline(ds['time'].values[t], color='black', ls='-', lw=lw_cut)

    #### - STRATOSPHERE - ####
    k=1
    # - Running mean - #
    # contf_st = axes[k,0].pcolormesh(ds['longitude2'], ds['geom_height'][t,:,0,0]/1000, ds['tmp_runningM'].sel(latitude=-53.75)[t,:,:], cmap=cmap_st, norm=norm_st, shading='nearest')
    # contf_st = axes[k,1].pcolormesh(ds.latitude,  ds['geom_height'][t,:,0,0]/1000, ds['tmp_runningM'].sel(longitude=360-67.75)[t,:,:], cmap=cmap_st, norm=norm_st, shading='nearest')
    
    ### FFT or T21 ###
    contf_st = axes[k,0].contourf(ds.longitude_plot, ds['level']/1000, tprime_lon_z, levels=clev, cmap=cmap_st, norm=norm_st, extend='both')
    contf_st = axes[k,1].contourf(ds.latitude, ds['level']/1000, tprime_lat_z, levels=clev, cmap=cmap_st, norm=norm_st, extend='both')

    # - Butterworth - #
    # contf_st = axes[k,0].contourf(ds['longitude'], ds['level']/1000, tprime_zonal.T, levels=clev, cmap=cmap_st, norm=norm_st, extend='both')
    # contf_st = axes[k,1].contourf(ds.latitude, ds['level']/1000, tprime_meridional.T, levels=clev, cmap=cmap_st, norm=norm_st, extend='both')

    # - THETA - #
    # cont  = axes[k,0].contour(ds.longitude, ds['geom_height'][t,:,0,0].values/1000, th_lon_z[t,:,:].values, colors='k') # , levels=thlev)
    # cont  = axes[k,1].contour(ds.latitude,  ds['geom_height'][t,:,0,0].values/1000, th_lat_z[t,:,:].values, colors='k', levels=thlev)
    cont_th0  = axes[k,0].contour(ds.longitude_plot, ds['level']/1000, th_lon_z[t,:,:], colors='dimgray', levels=thlev_st, linewidths=0.3)
    cont_th1  = axes[k,1].contour(ds.latitude,  ds['level']/1000, th_lat_z[t,:,:], colors='dimgray', alpha=1, levels=thlev_st, linewidths=0.3)
    th_y_pos = np.linspace(10,60,7)
    th_label_lon = []
    th_label_lat = []
    for lab in th_y_pos:
        th_label_lon.append((lon_label,lab))
        th_label_lat.append((lat_label,lab))
    axes[k,0].clabel(cont_th0, thlev_st_labels, fmt= '%1.0fK', inline=True, fontsize=fs_clabel, manual=th_label_lon)
    #axes[k,1].clabel(cont_th1, thlev_st_labels, fmt= '%1.0fK', inline=True, fontsize=9, manual=th_label_lat)
    # ax.clabel(isentropes, thlev[1::], fontsize=8, fmt='%1.0f K', inline_spacing=1, inline=True, 
    #             manual=[(8,ds.zcr[t,10,0,x]), (8,ds.zcr[t,-15,0,x])]) # ha='left', thlev[1::3]

    # - Wind u and v - #
    cont_v  = axes[k,0].contour(ds.longitude_plot, ds['level']/1000, v_lon_z[t,:,:], colors='k', levels=ulev, linewidths=lw_wind) # linestyles='-'
    cont_u  = axes[k,1].contour(ds.latitude,  ds['level']/1000, u_lat_z[t,:,:], colors='k', levels=ulev, linewidths=lw_wind)
    #cont_v  = axes[k,0].contour(ds.longitude_plot, ds['level']/1000, UV_lon_z[t,:,:], colors='k', levels=ulev, linewidths=0.9, linestyles='--')
    #cont_u  = axes[k,1].contour(ds.latitude,  ds['level']/1000, UV_lat_z[t,:,:], colors='k', levels=ulev, linewidths=0.9, linestyles='--')
    axes[k,0].clabel(cont_v, ulev, fmt= '%1.0f', inline=True, fontsize=fs_clabel)
    axes[k,1].clabel(cont_u, ulev, fmt= '%1.0f', inline=True, fontsize=fs_clabel)
    # th_y_pos = np.linspace(10,65,8)
    # v_label_lon = []
    # u_label_lat = []
    # for lab in th_y_pos:
    #     v_label_lon.append((lon_label+60,lab))
    #     u_label_lat.append((lat_label,lab))
    # axes[k,0].clabel(cont_v, ulev, fmt= '%1.0f', inline=True, fontsize=9, manual=v_label_lon)
    # axes[k,0].clabel(cont_u, ulev, fmt= '%1.0f', inline=True, fontsize=9, manual=u_label_lat)

    # - Coral position - #
    # axes[k,0].scatter(-67.75,-53.79, transform=ccrs.PlateCarree(), color="black",s=150, marker='x', lw=3)
    axes[k,0].axvline(lon, color='black', ls='--', lw=lw_cut)
    axes[k,1].axvline(lat, color='black', ls='--', lw=lw_cut)

    # - Draw rectangle around GWs above fold - #
    if plot_rectangle:
        rect_width = 20
        rect = Rectangle([-98,13],rect_width,43,linewidth=2, linestyle='dotted', edgecolor='black', facecolor='none')
        axes[k,0].add_patch(rect)
    
    axes[k,0].set_xlim(lon_range)
    axes[k,1].set_xlim(lat_range)
    axes[k,0].set_ylim(14,61)
    axes[k,1].set_ylim(14,61)
    
    # axes[k,0].text(0.73, 0.03, 'lidar', transform=axes[k,0].transAxes, weight='bold', color='black')
    axes[k,0].text(lon+3, 15, lidar_label, weight='bold', color='black')

    #### - TROPOSPHERE - ####
    # - Brunt-Vaisala frequency N^2 - #
    contf = axes[k+1,0].contourf(ds.longitude_plot, ds['level']/1000, n2_lon_z[t,:,:]*10**4, cmap=cmap, norm=norm, levels=n2lev, extend='both')
    contf = axes[k+1,1].contourf(ds.latitude,  ds['level']/1000, n2_lat_z[t,:,:]*10**4, cmap=cmap, norm=norm, levels=n2lev, extend='both')

    # - THETA - #
    # cont  = axes[k,0].contour(ds.longitude, ds['geom_height'][t,:,0,0].values/1000, th_lon_z[t,:,:].values, colors='k') # , levels=thlev)
    # cont  = axes[k,1].contour(ds.latitude,  ds['geom_height'][t,:,0,0].values/1000, th_lat_z[t,:,:].values, colors='k', levels=thlev)
    cont_th0  = axes[k+1,0].contour(ds.longitude_plot, ds['level']/1000, th_lon_z[t,:,:], colors='dimgray', levels=thlev, linewidths=0.3)
    cont_th1  = axes[k+1,1].contour(ds.latitude,  ds['level']/1000, th_lat_z[t,:,:], colors='dimgray', alpha=1, levels=thlev, linewidths=0.3)
    th_y_pos = np.linspace(6,14,3)
    th_label_lon = []
    th_label_lat = []
    for lab in th_y_pos:
        th_label_lon.append((lon_label,lab))
        th_label_lat.append((lat_label,lab))
    axes[k+1,0].clabel(cont_th0, thlev_labels, fmt= '%1.0fK', inline=True, fontsize=fs_clabel, manual=th_label_lon)
    #axes[k+1,1].clabel(cont_th1, thlev_labels, fmt= '%1.0fK', inline=True, fontsize=fs_clabel, manual=th_label_lat)

        # - Wind u and v - #
    cont_v  = axes[k+1,0].contour(ds.longitude_plot, ds['level']/1000, v_lon_z[t,:,:], colors='k', levels=ulev, linewidths=lw_wind)
    cont_u  = axes[k+1,1].contour(ds.latitude,  ds['level']/1000, u_lat_z[t,:,:], colors='k', levels=ulev, linewidths=lw_wind)
    axes[k+1,0].clabel(cont_v, ulev, fmt= '%1.0f', inline=True, fontsize=fs_clabel)
    axes[k+1,1].clabel(cont_u, ulev, fmt= '%1.0f', inline=True, fontsize=fs_clabel)

    # - PV - #
    # axes[k,0].contour(ds.longitude, ds['geom_height'][t,:,0,0]/1000, pv_lon_z[t,:,:], colors='k', lw=3, levels=pvlev)
    # axes[k,1].contour(ds.latitude,  ds['geom_height'][t,:,0,0]/1000, pv_lat_z[t,:,:], colors='k', lw=3, levels=pvlev)
    cont0 = axes[k+1,0].contour(ds_pv['longitude_3d'][t,:,0,:], zpv_lon_z[t,:,:]/(g*1000), pv_lon_z[t,:,:], colors=['k', 'k', 'limegreen', 'k'], linestyles='solid', linewidths=[1.5, 1.5, 2.5, 1.5], levels=pvlev)
    cont1 = axes[k+1,1].contour(ds_pv['latitude_3d'][t,:,:,0],  zpv_lat_z[t,:,:]/(g*1000), pv_lat_z[t,:,:], colors=['k', 'k', 'limegreen', 'k'], linestyles='solid', linewidths=[1.5, 1.5, 2.5, 1.5], levels=pvlev)
    axes[k+1,0].clabel(cont0, [-4,-3,-2,-1], fmt= '%1.0f', inline=True)
    axes[k+1,1].clabel(cont1, [-4,-3,-2,-1], fmt= '%1.0f', inline=True)

    # - Coral position - #
    # axes[k,0].scatter(-67.75,-53.79, transform=ccrs.PlateCarree(), color="black",s=150, marker='x', lw=3)
    axes[k+1,0].axvline(lon, color='black', ls='--', lw=lw_cut)
    axes[k+1,1].axvline(lat, color='black', ls='--', lw=lw_cut)

    # - Draw rectangle around GWs above fold - #
    if plot_rectangle:
        rect = Rectangle([-98,3.6],rect_width,30,linewidth=2, linestyle='dotted', edgecolor='black', facecolor='none')
        axes[k+1,0].add_patch(rect)

    axes[k+1,0].set_xlim(lon_range)
    axes[k+1,1].set_xlim(lat_range)
    axes[k+1,0].set_ylim(3,14)
    axes[k+1,1].set_ylim(3,14)

    axes[k,0].set_ylabel('altitude / km')
    axes[k+1,0].set_ylabel('altitude / km')

    for k in range(1,3):
        axes[k,0].yaxis.set_minor_locator(AutoMinorLocator()) 
        axes[k,0].xaxis.set_minor_locator(AutoMinorLocator())
        axes[k,1].yaxis.set_minor_locator(AutoMinorLocator()) 
        axes[k,1].xaxis.set_minor_locator(AutoMinorLocator())
        axes[k,0].tick_params(which='both', labelbottom=False,labeltop=False)
        axes[k,1].tick_params(which='both', labelbottom=False,labeltop=False,labelleft=False)

    # - Axis formatting - #
    k=2
    # axes[k,0].set_xlabel('longitude / °') # change to longitudes, latitude 10$^3$
    # axes[k,1].set_xlabel('latitude / °') # change to longitudes, latitude

    # axes[k,0].xaxis.set_major_formatter(FormatStrFormatter('%.0f°W'))
    axes[k,0].xaxis.set_major_formatter(major_formatter_lon)
    axes[k,1].xaxis.set_major_formatter(major_formatter_lat)
    axes[k,0].xaxis.set_label_position('bottom') 
    axes[k,1].xaxis.set_label_position('bottom')
    axes[k,0].tick_params(which='both', top=True, labelbottom=True,labeltop=False)
    axes[k,1].tick_params(which='both', top=True, labelbottom=True,labeltop=False)

    #### - 2PVU - ####
    z_levs = [3,4,5,6,7,8,9,10,11,12]
    geop_levels = np.arange(1000,4000,50)
    nbarbs = 25
    met_level = 700
    contf_pvu  = ax_pvu.contourf(ds_2pvu.longitude_plot, ds_2pvu.latitude, ds_2pvu['z'][t,:,:]/(1000*g), levels=z_levs, transform=projection,cmap='turbo', extend='both') # Spectral_r
    cont_met   = ax_pvu.contour(ds_pv.longitude_plot, ds_pv.latitude, ds_pv['z'].sel(level=met_level)[t,:,:]/g, transform=projection,colors='k', levels=geop_levels, linewidths=0.4)
    barbs_met  = ax_pvu.barbs(ds_pv.longitude_plot[::nbarbs], ds_pv.latitude[::nbarbs], ds_pv['u'].sel(level=met_level)[t,::nbarbs,::nbarbs], ds_pv['v'].sel(level=met_level)[t,::nbarbs,::nbarbs], transform=projection, length=5, linewidth=0.7)
    ax_pvu.clabel(cont_met, fmt= '%1.0fm', inline=True, fontsize=fs_clabel)

    # Lidar location
    #ax_pvu.scatter(lon,lat, transform=ccrs.PlateCarree(), facecolor="black",s=120, marker='x', lw=2.5)
    ax_pvu.axvline(lon, color='black', ls='--', lw=lw_cut)
    ax_pvu.axhline(lat, color='black', ls='--', lw=lw_cut)

    ax_pvu.coastlines()
    gls = ax_pvu.gridlines(draw_labels=True, x_inline=False, y_inline=False)
    gls.right_labels=False
    gls.top_labels=False
    
    # - Numbering - #
    # - Labels - #
    numb_str = ['a','b','c','d','e','f','g','h']
    k=0
    ypp = 0.9
    ax_lid.text(0.025, ypp, numb_str[0], transform=ax_lid.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    ax_lid2.text(0.17, ypp, numb_str[1], transform=ax_lid2.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})

    k=1
    ypp = 0.92
    axes[k,0].text(0.04, ypp, numb_str[2], transform=axes[k,0].transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    axes[k,1].text(0.04, ypp, numb_str[3], transform=axes[k,1].transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    k=2
    ypp = 0.90
    axes[k,0].text(0.04, ypp, numb_str[4], transform=axes[k,0].transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})
    axes[k,1].text(0.04, ypp, numb_str[5], transform=axes[k,1].transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})

    ax_pvu.text(0.03, 0.92, numb_str[6], transform=ax_pvu.transAxes, weight='bold', bbox={"boxstyle" : "circle", "lw":0.67, "facecolor":"white", "edgecolor":"black"})

    ax_pvu.set_xlim(lon_range)
    ax_pvu.set_ylim(lat_range)

    ax_lid.grid()

    # - COLORBARS - #
    #cbar = fig.colorbar(contf_st, ax=axes[0:2,2], location='right', ticks=clev_l, shrink=0.6, fraction=1, aspect=25)
    cbar = fig.colorbar(contf_st, ax=axes[0:2,2], location='right', ticks=clev_l, shrink=0.7, fraction=1, aspect=30)
    cbar.set_label(r"$T'$ / K")

    cbar = fig.colorbar(contf, ax=axes[2,2], location='right', shrink=0.9, fraction=1, aspect=15)
    cbar.set_label('$N^2$ / 10$^{-4}$ s$^{-2}$')

    cbar = fig.colorbar(contf_pvu, ax=axes[4,2], location='right', fraction=1, shrink=0.9, aspect=17)
    cbar.set_label('height of the dynamical tropopause / km')

    fig_name = folder + 'era5_trop_strat_' + '{:02d}'.format(t) + '.png'
    fig.savefig(fig_name, facecolor='w', edgecolor='w', format='png', dpi=300, bbox_inches='tight') # orientation='portrait'

##### - RUN - #####
#t  = 33 # 2018
t = 23 # 55 # 2020
#t = 41 # 2014
era5_cross_section_plot(ds,t,output_folder,lat,lon)

for tstep in range(np.shape(ds['t'])[0]):
    era5_cross_section_plot(ds,tstep,output_folder,lat,lon)

In [7]:
import imageio

image_folder = output_folder
filenames    = sorted(os.listdir(image_folder))
fps          = 4

with imageio.get_writer(image_folder + "/era5_sequence.gif", duration=1000*1/fps) as writer:
    for filename in filenames:
        if filename.endswith(".png"):
            image = imageio.imread(os.path.join(image_folder, filename))
            writer.append_data(image)

# imageio.mimsave(image_folder + "/era5_sequence.gif", images, duration=1/fps, palettesize=256/2)  # loop=0, quantizer="nq", palettesize=256
print("GIF created successfully!")

/var/folders/5n/r616r2rx3710m3kyf90_xwdr0000gn/T/ipykernel_72085/4056020747.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(os.path.join(image_folder, filename))


GIF created successfully!


In [56]:
####### Calculating wave properties #######
lambdaz = 9000;  Tground=10*3600 # m
#lambdaz = 20000; Tground=12*3600 # m
ctopo = 13.88 # m/s
N = 0.02
f = -1.195*10**(-4)

m       = -2*np.pi/lambdaz
lambdax = ctopo * Tground
k       = 2*np.pi/lambdax
phi     = np.arctan(lambdax/lambdaz)
cpz     = N/m * np.cos(phi)
cpx     = N/k * np.cos(phi)
alpha   = lambdaz/lambdax 
cgx     = N**2*alpha / (m * (f**2+N**2*alpha**2)**(1/2))
cgz     = -alpha * cgx
wintr  = (N**2 * k**2 / (k**2+m**2) + f**2 * m**2 / (k**2+m**2))**(1/2)
# round(cpz,2)
print("lambdax: ", lambdax)
print("phi: ", phi)
print("cpx: ", cpx)
print("cpz: ", cpz)
print("cgx: ", cgx)
print("cgz: ", cgz)
print("wintr: ", wintr)

lambdax:  499680.0
phi:  1.5527867467756002
cpx:  28.64324398260545
cpz:  -0.5159085731737294
cgx:  -27.190808748952612
cgz:  0.48974799619871423
wintr:  0.000379472768123237
